#### Aerosandbox Simulation Analysis Example

This example shows the basic CITROS data analysis functions and based on the simulations made with Aerosandbox python library.



First of all, we should import all neccesary libs:

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from prettytable import PrettyTable, ALL

import json

from platform import python_version

from citros import CitrosDB



Now we can connect to the simulation database:

In [ ]:
citros = CitrosDB(simulation='simulation_aerosandbox_cessna', batch='aerosandbox_cessna')

citros.info().print()


As you can see in the output above, we've got some information about our simulation run (batch):
1. Size of data
2. Sid information. In case we have more then one sid - this will be shown here.
3. Number and a list of topics. First three topics are build-in system topics, and the last one is our simulation results. 

Let's go further:

In [ ]:
citros.sid([0]).info().print()

This shows us more detailed statistics about topics: time stamps, durations, number of messages etc.
Let's inspect our data topic:

In [ ]:
citros.topic('/aerosandbox_cessna/state').sid(0).info().print()

Here we can see that our data topic contains 99 messages, and each one's type is a Float64MultiArray. 
Let's print all raw data:

In [ ]:
print(citros.topic('/aerosandbox_cessna/state').data())

To get all json-data columns for a specific topic, function data() is used. It returns pandas DataFrame. To get a specific columns, pass their labels in a list as an argument:

In [ ]:
print(citros.topic('/aerosandbox_cessna/state').data(['data.data[0]','data.data[1]'])[['data.data[0]', 'data.data[1]']])

Now we can create some plots. Let's plot coordinates along one of the axes (supposing messages come once a second): 

In [ ]:
fig1, ax1 = plt.subplots()



citros.time_plot(ax1, 

                 topic_name = '/aerosandbox_cessna/state', 

                 var_name = 'data.data[0]', 

                 time_step = 1, 

                 sids = [0], 

                 y_label = 'X coords', title_text = 'X coords vs. Time')

Let's do some simple research. This simulation was started with custom parameter setup: one of the parameters (`h_0`) was defined with user-defined function, so we will see some difference between sid's data:

In [ ]:
citros.info().print()

You can see here more than one sids - the simulation batch settings have been adjusted in such a way that each simulation corresponds to a parameter value from the range [1000..10000].

We will use this CITROS advantage to analyze how initial altitude affects on the gliding distance:

In [ ]:
#create a figure to plot on

fig2, ax2 = plt.subplots()



citros.xy_plot(ax2, 

               topic_name = '/aerosandbox_cessna/state', 

               var_x_name = 'data.data[0]',

               var_y_name = 'data.data[1]',

               sids = [0,1,2,3,4,5,6,7,8,9], 

               x_label = 'x', y_label = 'y', title_text = 'Gliding trajectory')

This plot shows us that the trajectories have different gliding distance depending on initial altitude. You can see the small parts of plots where our Cessna trying to reach maximum altitude and optimal velocity.

Let's investigate how powerfull the influence is:

In [ ]:
# Defining the list of altitudes

h_0 = [i for i in range(1000,11000, 1000)]


# Setting Dataframe

df = citros.topic('/aerosandbox_cessna/state').set_order({'sid':'asc'}).data('data.data[0]')
sid_list = list(set(df['sid']))
data0_list = []
for s in sid_list:
    id_max = df[df['sid'] == s]['rid'].idxmax()
    data0_list.append(df['data.data[0]'].loc[id_max])

fig, ax = plt.subplots()

c = np.random.choice(50, 10, replace=False)
scatter = ax.scatter(h_0, data0_list,c=c)

# Create legend entries for each point
legend_labels = [str(i) for i in range(10)]

# Initialize a list to store legend handles
legend_handles = []

# Loop through the points and create legend entries with matching colors
for i, label in enumerate(legend_labels):
    color = scatter.to_rgba(c[i])  # Get the color of the corresponding point
    legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10))

# Add the legend with custom handles
legend1 = ax.legend(handles=legend_handles, loc="upper left", title="sid")
ax.add_artist(legend1)
ax.grid()
# plt.scatter(h_0, data0_list, cmap='plasma')
# # plt.plot(h_0, data0_list)
ax.set_ylabel('Gliding distance, m')
ax.set_xlabel('Initial altitude, m')
ax.set_title('Maximum gliding distance vs Initial altitude')
# plt.legend()


The gliding disctance increases from ~21000 m up to 187000 m while the initial altitude rises from 1000 to 10000. 